# CBF Visibility Capture

In [ ]:
# This part of the notebook requires the visibilities be captured and saved as a panda dataframe.
# This can be done by starting a pod deploying ska-mid-cbf-signal-verification-visibility-capture at the receiving end,
# and run this command:
#
#   python capture.py -c vis_capture_cfg_temp.json -o /app/data/visibility_capture.zip --band {frequency_band} --slice {frequency_slice} -t 30 -d
#
# where frequency_band should match configure scan - common - frequency_band, and
# frequency_slice_id should match configure scan - cbf - fsp - frequency_slice_id

# Path to Home Directory
home_dir = "/home/ja097202_p"

# Path to Kubeconfig
path_to_kubeconfig = "/home/ja097202_p"

# Edit this path as needed
vis_zip_file = f"{home_dir}/visibility_capture.zip"

# Delete the file if it already exists from a previous run
!rm $vis_zip_file

# Adjust this depending on the number of receptors.
# TODO: Update this when baselines are actually labelled by receptor IDs
baselines = [0]

# psi or itf
ENVIRONMENT = "psi"

In [ ]:
# Setup pod and capture data
import time

if ENVIRONMENT == "itf":
    !kubectl --kubeconfig=$path_to_kubeconfig/KUBECONFIG apply -f visibilities_itf.yaml
elif ENVIRONMENT == "psi":
    !kubectl apply -f visibilities_psi.yaml
else:
    print(
        "Please update your environment variable or specify which visibilities yaml you intend to run."
    )
time.sleep(5)

In [ ]:
mkdir_command = "'mkdir -p /app/data && chmod 777 /app/data && python capture.py -c vis_cfg.json -o /app/data/visibility_capture.zip --band 1 --slice 1 -t 60 -d'"
if ENVIRONMENT == "itf":
    !kubectl --kubeconfig=$path_to_kubeconfig/KUBECONFIG cp vis_cfg.json visibilities-emulator:/app/vis_cfg.json
    time.sleep(5)
    !kubectl --kubeconfig=$path_to_kubeconfig/KUBECONFIG exec visibilities-emulator -- /bin/bash -c $mkdir_command
elif ENVIRONMENT == "psi":
    !kubectl cp vis_cfg.json visibilities-emulator:/app/vis_cfg.json
    time.sleep(5)
    !kubectl exec visibilities-emulator -- /bin/bash -c $mkdir_command
else:
    print(
        "Please update your environment variable or specify which visibilities yaml you intend to run."
    )

In [ ]:
# Copy visibilities zip file out of pod
if ENVIRONMENT == "itf":
    !kubectl --kubeconfig=$path_to_kubeconfig/KUBECONFIG cp visibilities-emulator:/app/data/visibility_capture.zip $vis_zip_file
elif ENVIRONMENT == "psi":
    !kubectl cp visibilities-emulator:/app/data/visibility_capture.zip $vis_zip_file
else:
    print(
        "Please update your environment variable or specify which visibilities yaml you intend to run."
    )

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import Markdown


def add_polar(df):
    for pol in ("XX", "XY", "YX", "YY"):
        cplx = df[f"vis_{pol}_real"] + 1j * df[f"vis_{pol}_imag"]
        df[f"vis_{pol}_mag"] = np.abs(cplx)
        df[f"vis_{pol}_phase"] = np.angle(cplx)
    return df


def load(filepath):
    load_data = pd.read_pickle(filepath, compression="gzip")
    return load_data


PLOT_TIME_SPAN = 10

# Max number of points to plot
SV_CONTAINER_MAX = 100000

N_CHAN = 14880

In [ ]:
from IPython.display import display

p = Path(vis_zip_file)
data = load(p)
data = add_polar(data)

n_baselines = data.index.get_level_values(level="baseline").nunique()
print(f"Number of baselines = {n_baselines}")

display(Markdown("**FD by baselines**"))
df_filtered = data.groupby(level=["baseline"])["FD"].value_counts()
print(df_filtered)

display(Markdown("**Timestamps with FD < 1**"))
df_filtered = data[data["FD"] < 1.0].groupby(level=["timestamp"])["FD"].value_counts()
print(df_filtered)

timestamps = data.index.unique(level="timestamp")
start_time = np.min(timestamps)
end_time = np.max(timestamps)

# TRUNCATE TIME FOR PLOTTING
plot_start_time = start_time
if (end_time - start_time) > PLOT_TIME_SPAN:
    # the truncation is done because the signal-verification container doesn't have
    # the resources available to plot the entire produced lstv. plot subset
    plot_end_time = plot_start_time + PLOT_TIME_SPAN
    print(f"Truncating data to first {PLOT_TIME_SPAN} seconds of timestamps")
else:
    plot_end_time = end_time

plot_timestamps = timestamps[(timestamps >= plot_start_time) & (timestamps <= plot_end_time)]
plot_timestamps = pd.Index(
    [
        plot_timestamps[0],
        plot_timestamps[plot_timestamps.size // 3],
        plot_timestamps[plot_timestamps.size // 3 * 2],
        plot_timestamps[-1],
    ]
)
print(f"Timestamps to plot are : {plot_timestamps}")

subarrays = np.unique(data.index.get_level_values("subarray"))

In [ ]:
import plotly.graph_objects as go

plot_columns = [
    "vis_XX_mag",
    "vis_XX_phase",
    "vis_YY_mag",
    "vis_YY_phase",
    "vis_XY_mag",
    "vis_XY_phase",
]

# plot figure for each of the above columns
for baseline in baselines:
    for col in plot_columns:
        fig = go.Figure()

        for timestamp in plot_timestamps:
            vis_df = data.loc[
                slice(None),
                slice(None),
                slice(baseline, baseline),
                slice(timestamp, timestamp),
            ]

            vis_df.reset_index(level="baseline", drop=True, inplace=True)
            vis_df.reset_index(level="timestamp", drop=False, inplace=True)

            fig.add_trace(
                go.Scatter(
                    x=vis_df["sky_frequency"],
                    y=vis_df[col],
                    mode="lines",
                    name=f"time_{timestamp:.2f}_{col}",
                )
            )

        fig.update_layout(
            title=f"Plot of {col}, from {plot_timestamps[0]:.3f} to {plot_timestamps[-1]:.3f} seconds for baseline {baseline}."
        )
        fig.update_xaxes(
            title_text="Sky Frequency (Hz)",
        )
        if col.endswith("mag"):
            fig.update_yaxes(title_text="Frequency response (20.log10(abs(H)))", type="log")
        elif col.endswith("phase"):
            fig.update_yaxes(title_text="Phase")

        fig.show()

In [ ]:
subarrays = np.unique(data.index.get_level_values("subarray"))

integrated_df = (
    data.loc[slice(None), slice(None), baselines, plot_timestamps.to_list()]
    .groupby(level=("subarray", "channel", "baseline"))
    .mean()
)  # integrate and normalise.

fig = go.Figure()
for subarray in subarrays:
    for baseline in baselines:
        vis_df = integrated_df.loc[
            slice(subarray, subarray),
            slice(None),
            slice(baseline, baseline),
        ].copy()

        vis_df.reset_index(level="subarray", drop=True, inplace=True)
        vis_df.reset_index(level="baseline", drop=True, inplace=True)

        plot_columns = ["vis_XX_mag", "vis_YY_mag", "vis_YX_mag"]

        for col in plot_columns:
            fig.add_trace(
                go.Scatter(
                    x=vis_df["sky_frequency"],
                    y=vis_df[col],
                    mode="lines",
                    name=f"sa{subarray}_{col}_{baseline}",
                )
            )

    fig.update_layout(
        title=f"Visibility Spectrum, integrated over {plot_end_time-plot_start_time:.3f} seconds for baselines {baselines}."
    )
    fig.update_xaxes(
        title_text="Sky Frequency (Hz)",
    )
    fig.update_yaxes(title_text="Frequency response (20.log10(abs(H)))", type="log")

    fig.show()